In [32]:
import process_gens
import pandas as pd
from helm.benchmark.runner import InstanceGenerations,GenerationSummary
from typing import Any, List
import json
from helm.common.request import (GeneratedOutput, Token)

import PostMetric
import pandas as pd

from helm.benchmark.metrics.statistic import Stat
from typing import Dict, Optional

from helm.benchmark.augmentations.perturbation_description import (
    PerturbationDescription)
from dataclasses import dataclass
from process_gens import ProcessGens
# from process_gen_utils import *

In [33]:
def get_metrics(mode):
    if(mode=="wmt"):
        task_name="wmt"
        custom_metrics=[ PostMetric.BLEU1_METRIC(),PostMetric.BLEU4_METRIC()]
        instance_metrics=["comet"]
    if(mode=="instruct"):
        task_names=["open_assistant:language=en,num_respondents=1, self_instruct:num_respondents=1,"]
        custom_metrics=[]
        instance_metrics=[]
    return task_names, custom_metrics, instance_metrics
    

In [ ]:

# # copied snellius test
# num_beams_list=[32, 64]
# models=["meta_llama_Llama_3.1_8B_Instruct"]
# eval_instances=5
# task_name="wmt_14_language_pair_de_en_"
# root_folder=f"snellius_copies/wmt_test"
# instance_metrics=["comet"]

# #local test
# num_beams_list=[15,20]
# root_folder="helm_output"
# eval_instances=3
# models=["distilbert_distilgpt2"]

# copied snellius run
# num_beams_list=[2,4,8,16,32,64,128]
# num_beams_list=[2,64]

# models=["meta_llama_Llama_3.1_8B_Instruct"]
# eval_instances=1000
# root_folder=f"snellius_copies/full"


In [34]:
# #always true
# root_folder="snellius_copies/helm_output"

# #suite stuff
# mode="instruct"
# suite_name="instruct_2_evals"
# #specifics
# num_beams_list=[1]
# models=["allenai_OLMo_2_1124_13B_Instruct"]
# eval_instances=2



####################### For test #######################
#always true
root_folder="helm_output"

#suite stuff
mode="instruct"
suite_name="instruct_1_evals"

#specifics
num_beams_list=[1]
models=["allenai_OLMo_2_1124_13B_Instruct"]
eval_instances=2


test_process_gens=ProcessGens()

task_names, custom_metrics, instance_metrics= get_metrics(mode)
test_process_gens.init(root_folder=root_folder,num_beams_list=num_beams_list,models=models,custom_metrics=custom_metrics,task_names=task_names, eval_instances=eval_instances, suite_name=suite_name,instance_metrics=instance_metrics)


hello
Analyzing path: snellius_copies/helm_output/instruct_2_evals/open_assistant_language_en_num_respondents_1_/allenai_OLMo_2_1124_13B_Instruct/1_beams/runs/instruct_2_evals/per_instance_stats.json
Getting gen summary from: snellius_copies/helm_output/instruct_2_evals/open_assistant_language_en_num_respondents_1_/allenai_OLMo_2_1124_13B_Instruct/1_beams/runs/instruct_2_evals/generation_summary_metrics.json
number of instances: 2
get_metrics_df
get_instance_info
Getting gen summary from: snellius_copies/helm_output/instruct_2_evals/open_assistant_language_en_num_respondents_1_/allenai_OLMo_2_1124_13B_Instruct/1_beams/runs/instruct_2_evals/generation_summary_metrics.json
Analyzing path: snellius_copies/helm_output/instruct_2_evals/open_assistant_language_en_num_respondents_1_/allenai_OLMo_2_1124_13B_Instruct/1_beams/runs/instruct_2_evals/per_instance_stats.json
calculate_beam_num_to_summary
Getting gen summary from: snellius_copies/helm_output/instruct_2_evals/open_assistant_language_e

In [ ]:
# tests
# beam_num_to_instance_stats=test_process_gens.calculate_beam_num_to_run_instance_stats(root_folder=root_folder, num_beams_list=num_beams_list, models=models, task_name=task_name, suite_name=suite_name, instance_metrics=instance_metrics)
# beam_num_to_summary=test_process_gens.calculate_beam_num_to_summary(root_folder=root_folder, num_beams_list=num_beams_list, models=models,eval_instances=eval_instances,task_name=task_name, suite_name=suite_name)

In [35]:


# beam_num=next(iter(processGens.beam_num_to_summary))
# instance_generations_by_id=get_instance_generations_by_id()
# get_beam_means()
# get_beam_probs()
# check_completion_logprob(beam_num)
# check_sentence_logprob(beam_num)
# see_overlap_per_instance_generation(num_beams_list[-1], num_instances=10)

# if(processGens.metrics_df is not None):
#     ax1 = processGens.metrics_df.plot.scatter(x='beam_num',y='comet',c='DarkBlue')

last_beam=num_beams_list[-1]
first_beam=num_beams_list[0]
compare_beams_by_metric(analysis_df=completions_df,compare_metric='output_logprob', compare_beams=[last_beam, first_beam])
compare_beams_by_metric(analysis_df=completions_df,compare_metric='comet', compare_beams=[last_beam, first_beam])
plot_keys("completion_logprob","BLEU")







NameError: name 'completions_df' is not defined